# Natural constraints


In [ ]:
# standard
import pickle
from collections import defaultdict

# third party
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# first party
from config import Config

## Read in data

The results are the output of `02_natural_constraints.py`, and should be stored in `./results/natural_constraints/`.

In [ ]:
def to_dataframe(a_dict):
    out = []
    for key, val in a_dict.items():
        if val.data is None:
            continue
        out.append(pd.DataFrame({'x': val.values, 'loc': val.geo_value, 'dates': val.dates}))
    out = pd.concat(out)
    out.set_index(['loc', 'dates'], inplace=True)
    return out

truth = to_dataframe(pickle.load(open('../data/tf_ground_truths.p', 'rb')))

In [ ]:
storage_dir = './results/natural_constraints/'
as_of_date_range = Config.every_10_as_of_range

output = defaultdict(list)
for as_of in as_of_date_range:
    print(as_of)
    result = pickle.load(open(f'{storage_dir}/as_of_{as_of}.p', 'rb'))
    if len(result.keys()) != 2:
        print('Not all options ran on', as_of, 'skipping')
        continue 
    for method, method_data in result.items():
        predictions = to_dataframe(method_data)
        errors = (truth - predictions).dropna().reset_index()
        errors['as_of'] = as_of
        output[method].append(errors)

In [ ]:
analysis = []
for method, method_data in output.items():
    all_errors = pd.concat(method_data)
    all_errors['method'] = method
    all_errors['abs_err'] = np.abs(all_errors.x)
    all_errors['lag'] = (pd.to_datetime(all_errors.as_of) - all_errors.dates).dt.days 
    analysis.append(all_errors)
    
analysis = pd.concat(analysis, ignore_index=True)

In [ ]:
analysis.replace(
    {'tf': 'Trend filtering', 'ntf': 'Natural trend filtering'},
    inplace=True
)

In [ ]:
# Save to pickle.
pickle.dump(analysis, open('natural_constraints_02_analysis.p', 'wb'))

In [ ]:
plt.figure(figsize=(10, 5))
sns.lineplot(
 data=analysis,
 x='lag',
 y='abs_err',
 hue='method',
 style='method',
 markers=True,
 err_kws={'alpha': 0.1}
)
plt.title('Effect of natural constraints in deconvolution')
plt.ylabel('Mean absolute error')
plt.xlabel('Days back from prediction time')
plt.tight_layout()

In [ ]:
plt.figure(figsize=(5, 5))
sns.lineplot(
 data=analysis[analysis.lag.le(10)],
 x='lag',
 y='abs_err',
 hue='method',
 style='method',
 markers=True,
 err_kws={'alpha': 0.1}
)
plt.title('Effect of natural constraints in deconvolution')
plt.ylabel('Mean absolute error')
plt.xlabel('Days back from nowcast time')
plt.xticks(range(2, 11, 2))
plt.legend(title=None)
plt.tight_layout()
plt.savefig('./figures/natural_constraints_02_small_square.pdf')